In [1]:
%matplotlib inline

# To reload external scripts automatically
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
import sys
sns.set_context('notebook')

from IPython.display import Image
from IPython.core.display import HTML 
from IPython.display import display
from amazon.api import AmazonAPI

# Importing external files
sys.path.append('scripts/')
from data_import import *
from similarities import *
from utils import *
from amazon_api_interaction import *


# 0-Exploratory
This notebook contain the first study of the data to identify what is needed and what we do not need. It will only work with subsets of datasets as they will not fit into memory.

In [2]:
DATA_FOLDER = "../../Project-Data/"
META_FOLDER = DATA_FOLDER + "meta/"
REVIEWS_FOLDER = DATA_FOLDER + "reviews/"
CORE_FOLDER = DATA_FOLDER + "5_core/"
DUMP_FOLDER = DATA_FOLDER + "dump/"
CATEGORIES = ['Books','Movies_and_Tv','Electronics']
MAXCOUNT = -1

## 0.0 - Content of the files

The different columns of the **metadata** files are : 
* ```asin``` : the unique identifier of the object
* ```brand```
* ```categories``` : the categories of the object
* ```description``` : the description of the object
* ```imUrl```  : the link toward the images related to the object
* ```price```
* ```related``` : a list of objects that are related to this object
* ```salesRank``` 
* ```title```

The different columns of the **reviews** and **5-core** files are :
* ```asin``` : the unique identifier of the object
* ```helpful``` : a list of 2 integers [x,y], the helpfulness score is x/y votes
* ```overall``` : the rating of the object
* ```reviewText```
* ```reviewTime```
* ```reviewerID```
* ```reviewerName```
* ```summary ``` : the title of the review
* ```unixReviewTime``` : in Unix format

Therefore we keep only the column that are of interest for our task

In [3]:
meta_interesting_cols = ['asin', 'title', 'salesRank', 'description','imUrl']
review_interesting_cols = ['asin', 'overall', 'unixReviewTime']

## 0.1-Books

In [4]:
meta_books_path, review_books_path, core_book_path = get_paths(0, DATA_FOLDER, META_FOLDER,CORE_FOLDER,
                                               REVIEWS_FOLDER, CATEGORIES)

Paths : 
	 meta = ../../Project-Data/meta/meta_Books.json
	 review = ../../Project-Data/reviews/reviews_Books.json
	 core_path = ../../Project-Data/5_core/Books.json


### 0.1.1-Metadata

In [5]:
meta_books = import_interesting_cols(meta_books_path,DUMP_FOLDER,True,meta_interesting_cols,max_count=MAXCOUNT)
meta_books.head(1)

Retrieving from : ../../Project-Data/dump/meta_Books_asin_title_salesRank_description_imUrl_ALL
It took 00:00:04.165 to import the data.


,asin,description,imUrl,"salesRank_Arts,_Crafts_&_Sewing",salesRank_Books,salesRank_Cell_Phones_&_Accessories,salesRank_Clothing,salesRank_Electronics,salesRank_Health_&_Personal_Care,salesRank_Home_&_Kitchen,...,salesRank_Jewelry,salesRank_Kitchen_&_Dining,salesRank_Movies_&_TV,salesRank_Music,salesRank_Musical_Instruments,salesRank_Office_Products,salesRank_Shoes,salesRank_Sports_&_Outdoors,salesRank_Toys_&_Games,title
0,0001048791,NaN,http://ecx.images-amazon.com/images/I/51MKP0T4...,NaN,6334800.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"The Crucible: Performed by Stuart Pankin, Jero..."


### 0.1.2-Reviews

In [6]:
review_books = import_interesting_cols(
    review_books_path, DUMP_FOLDER,False, review_interesting_cols, max_count=MAXCOUNT)
review_books.head()

Processing the JSON -

KeyboardInterrupt: 

### 0.1.3 - 5-Core

In [7]:
core_books = import_interesting_cols(
    core_book_path,
    DUMP_FOLDER,
    False,
    review_interesting_cols,
    max_count=MAXCOUNT)
core_books.head()

Retrieving from : ../../Project-Data/dump/Books_asin_overall_unixReviewTime_ALL
It took 00:00:00.540 to import the data.


,asin,overall,unixReviewTime
0,000100039X,5.0,2012-12-16
1,000100039X,5.0,2003-12-11
2,000100039X,5.0,2014-01-18
3,000100039X,5.0,2011-09-27
4,000100039X,5.0,2002-10-07


## 0.2-Movies and TV

In [ ]:
meta_movie_path, review_movie_path, core_movie_path = get_paths(
    1, DATA_FOLDER, META_FOLDER, CORE_FOLDER, REVIEWS_FOLDER, CATEGORIES)

### 0.2.1-Metadata

In [ ]:
meta_movie = import_interesting_cols(
    meta_movie_path,DUMP_FOLDER, True, meta_interesting_cols, max_count=MAXCOUNT, dropna=False)
meta_movie.head()

### 0.2.2-Reviews

In [ ]:
review_movie = import_interesting_cols(
    review_movie_path,
    DUMP_FOLDER,
    False,
    review_interesting_cols,
    max_count=100,
    dropna=False)
review_movie.head()

## 0.3-Electronics

In [ ]:
meta_electronic_path, review_electronic_path, core_electronic_path = get_paths(
    2, DATA_FOLDER, CORE_FOLDER, META_FOLDER, REVIEWS_FOLDER, CATEGORIES)

### 0.3.1-Metadata

In [ ]:
meta_electronic = import_interesting_cols(
    meta_electronic_path,
    DUMP_FOLDER,
    True,
    meta_interesting_cols,
    max_count=MAXCOUNT,
    dropna=False)
meta_electronic.head(2)

### 0.3.2-Review

In [ ]:
review_electronic = import_interesting_cols(
    review_electronic_path,
    DUMP_FOLDER,
    False,
    review_interesting_cols,
    max_count=MAXCOUNT,
    dropna=False)
review_electronic.head()

# 1-Detecting similar products
In this section we try to obtain the products that are similar among a given category and to develop a systematic way to detect those similar products.

## 1.1-Books

### 1.1.0 - Defining the dataset to work with

Some books are sold in different formats : hard cover, pocket, electronic, etc. Therefore in order to compare them we will focus on the title and description that are the only two attributes that should be highly similar between the two different yet similar books.

In [8]:
core_asin = list(core_books['asin'].unique())

# We delete all the rows where either the title or description is null and keep only core books
core_meta_df = meta_books[meta_books['title'].notnull()]
core_meta_df = core_meta_df[core_meta_df['description'].notnull()]
core_meta_df = core_meta_df[core_meta_df['imUrl'].notnull()]
core_meta_df = core_meta_df[core_meta_df['asin'].isin(core_asin)]
book_desc_titles = core_meta_df[['asin','title','description','imUrl']]
book_desc_titles = book_desc_titles.set_index(['asin'])
book_desc_titles.head()

,title,description,imUrl
asin,,,
0001055178,Master Georgie,Beryl Bainbridge seems drawn to disaster. Firs...,http://ecx.images-amazon.com/images/I/51ZSC6TK...
000171287X,The Berenstains' B Book (Bright & Early Books),"By Stan Berenstain and Jan Berenstain, Illustr...",http://ecx.images-amazon.com/images/I/31hjUxZu...
000100039X,The Prophet,"In a distant, timeless place, a mysterious pro...",http://ecx.images-amazon.com/images/I/81ZKLPiv...
0001473905,Rightly Dividing the Word,--This text refers to thePaperbackedition.,http://ecx.images-amazon.com/images/I/61KPC59B...
0001714538,The Berenstain Bears on the Moon (Bright and E...,PreSchool-Grade 2 A delightful tale told in rh...,http://ecx.images-amazon.com/images/I/A1zj84uC...


In [9]:
book_desc_titles.index.is_unique

True

We first try to experiment with tf-idf weights to find common titles among books. Due to the large size of the database, this task cannot be executed on all the books, we decided to focus as suggested by the dataset on the 5-core dataset. The 5 core dataset contains only products and reviews such that reviewers and products have at least 5-reviews. This will allow us in a first analysis to reduce the number of products that we consider. Since we will need to have at least 5 reviews anyway to look at the **Herding effect**.

Let's explore a bit what the dataset contains. we have much informations (and in an heterogeneous manner) in the description. Therefore we might want to observe the name of the auhtor and the title when they are available.

### 1.1.1 - Locality sensitive hashing
Based on this [article](http://dataconomy.com/2017/06/locality-sensitive-hashing-pydata/), and following the theory that can be read at the chapter 3.4 of [Mining of massive datasets by Leskovec, Rajaraman and Milliway](http://infolab.stanford.edu/~ullman/mmds/book.pdf) we will try to find similar books.

#### The ```minhash```

We need to detect near similar books in this large datasets. To do so we would need to compare each of the attribute of a given book with all the others which is extremely computationaly expensive. There we will use LSH. Right now the title and description of the book have variable length depending on the books. To perform LSH we would like to have a fixed length representation of the document without modifying the semantics of document similarity.
* first we introduce the principle of shingles. A shingle of 5-gram (hence we discard the last shingle as it only consists in the $<5$ last characters of the document) e.g. is a set of all possible 5-grams in the string. 

In [10]:
document = "Lorem Ipsum dolor sit amet"
shingles = get_shingles(document)

In [11]:
other_document = 'Lorem Ipsum dolor sit amet with some extra garbage'
other_shingles = get_shingles(other_document)
jaccard_dist(shingles,other_shingles)

0.4666666666666667

Each document will still have a different number of shingles depending on its length. Therefore we wish to represent it using a *fixed length representation*. We will present now a function ```minhash``` that has a collision probability that is exactly the jaccard similarity (based on this [document](http://infolab.stanford.edu/~ullman/mmds/ch3.pdf) and following this [repo](https://github.com/mattilyra/LSH/blob/master/examples/Introduction.ipynb) . We explain this now :
1. Suppose that the following dataframe is the boolean variable corresponding to whether or not a shingle belongs to a document
2. the ```minhash``` of a document returns a random permutation of the rows and then the first row number it founds where the value is non 0.
    * therefore row 1 for doc 1
    * and row 0 for doc 2

In [12]:
ex_minhash = pd.DataFrame({'shingleID':[1,2,3,4,5,6],'doc1':[0,1,0,0,1,1],'doc2':[1,1,1,0,1,0]})
ex_minhash

,doc1,doc2,shingleID
0,0,1,1
1,1,1,2
2,0,1,3
3,0,0,4
4,1,1,5
5,1,0,6


But this is for this particular permutation, if we had another one like below then the ```minhash``` would return :
   * row 2 for doc1
   * row 1 for doc2

There are a lot of such permutations. Nevertheless we only care about having the same row for each document which mean we have collision of the ```minhash```. Therefore we have only two rows for which this happens : $\text{shingleID} \in \{2,5\}$. Hence the probability that the two doc have the same ```minhash``` (remmember that rows with two zeros do not count as they are not considered by ```minhash```) is the number of rows where both doc value is 1 divided by the number of rows where they are different : $\frac{2}{5}$

In [13]:
ex_minhash.sample(frac=1)

,doc1,doc2,shingleID
3,0,0,4
2,0,1,3
1,1,1,2
5,1,0,6
0,0,1,1
4,1,1,5


We can chose the length of the fingerprint that is returned by the ```minhash```, as we increase the length we get less variance from the random initialisation of the ```minhasher``` but this also greatly increases the memory usage.

#### The principle of LSH

In [14]:
from lsh import minhash
hasher = minhash.MinHasher(seeds=100, char_ngram=5)
fingerprint0 = hasher.fingerprint('Lorem Ipsum dolor sit amet'.encode('utf8'))
len(fingerprint0)

100

Therefore the hash of a document is composed of a given (```seeds```) number of ```minhashes```. We divide this number of ```minhashes``` into a given number of parts (e.g. 5). Since every single ```minhash``` has the jaccard similarity probability of collision then each of the 5 parts will have this probability as well. These parts represent the **locality** in the term LSH.

Then we hash the content of each part using a different hash function to obtain the *binID* that represents the **hashing** part of the method. Into each bin with *binID* we store the entire fingerprint of the document.

The idea is then that we compare documents that fall in the same bins : we will compare their fingerprint which is equivalent to looking at their Jaccard similarity between shingle sets. Since not all documents will fall into the same bins we have reduced the number of potential candidates.

Indeed we call :
* ```seeds``` :  number of ```minihashes``` that compose the fingerprint
* ```bands``` : the number of bins that we want to use

#### Finding candidate duplicates

In [15]:
book_desc_titles.head()

,title,description,imUrl
asin,,,
0001055178,Master Georgie,Beryl Bainbridge seems drawn to disaster. Firs...,http://ecx.images-amazon.com/images/I/51ZSC6TK...
000171287X,The Berenstains' B Book (Bright & Early Books),"By Stan Berenstain and Jan Berenstain, Illustr...",http://ecx.images-amazon.com/images/I/31hjUxZu...
000100039X,The Prophet,"In a distant, timeless place, a mysterious pro...",http://ecx.images-amazon.com/images/I/81ZKLPiv...
0001473905,Rightly Dividing the Word,--This text refers to thePaperbackedition.,http://ecx.images-amazon.com/images/I/61KPC59B...
0001714538,The Berenstain Bears on the Moon (Bright and E...,PreSchool-Grade 2 A delightful tale told in rh...,http://ecx.images-amazon.com/images/I/A1zj84uC...


In [25]:
dump_path = DUMP_FOLDER
get_candidate = candidate_duplicates(book_desc_titles,dump_path,['title'])

Found 6187 items with possible duplicates.


In [26]:
# Let's quickly oberve the type of data that we obtain
element_to_inspect = 67
similarity_keys = list(get_candidate.keys())
print(get_titles(similarity_keys[element_to_inspect],get_candidate,book_desc_titles))
print(get_desc(similarity_keys[element_to_inspect],get_candidate,book_desc_titles))
display_images(similarity_keys[element_to_inspect],get_candidate,book_desc_titles)

['Hidden', 'Hidden', 'Hidden', 'Hidden']
['"Many teen readers will identify with Wren and Darra and how events that happened to us when we were younger help shape the person we become." —VOYA\xa0"Beginning with a horrific story of an accidental kidnapping, this poetic novel is impossible to put down....A masterpiece!"\xa0 --Shirley Mullin, Kids Ink Children\'s Bookstore\xa0“Like Frost’s Printz Honor Book, Keesha’s House (2003), this novel in verse stands out through its deliberate use of form to illuminate emotions and cleverly hide secrets in the text.” —Booklist', 'San Francisco fashion designer Kate Whitehead finally realized she had to get away from her abusive husband, California state senator Thomas Whitehead, not only for her own safety but also for the sake of her unborn child. So one day Kate "disappeared," and shortly thereafter Tricia Campbell turned up in San Diego. With little money and no one to turn to, Tricia is desperate, but fortunately she meets firefighter Scott McC

We can see that the books can greatly vary eventhough they have the same title. Also it seems hard to extract any meaningful data from their description therefore we need to use the Amazon API to try to get the data we are looking for.

#### Need to use Amazon API to get structured data
Because of the heterogeneous nature of the data in description we use Amazon API to obtain consistent informations like the name of authors, ISBN etc...
We use a [wrapper](https://pypi.python.org/pypi/python-amazon-simple-product-api) of the amazon API and the documentation can be found [here](http://pythonhosted.org/python-amazon-simple-product-api/).

Also note that given the ASIN of a product one can access the page of the product using one of the two following addresses:
* ```https://www.amazon.co.uk/dp/<ASIN>/```
* ```https://www.amazon.com/dp/<ASIN>/```


*Still in construction*

In [18]:
book_desc_titles.head()

,title,description,imUrl
asin,,,
0001055178,Master Georgie,Beryl Bainbridge seems drawn to disaster. Firs...,http://ecx.images-amazon.com/images/I/51ZSC6TK...
000171287X,The Berenstains' B Book (Bright & Early Books),"By Stan Berenstain and Jan Berenstain, Illustr...",http://ecx.images-amazon.com/images/I/31hjUxZu...
000100039X,The Prophet,"In a distant, timeless place, a mysterious pro...",http://ecx.images-amazon.com/images/I/81ZKLPiv...
0001473905,Rightly Dividing the Word,--This text refers to thePaperbackedition.,http://ecx.images-amazon.com/images/I/61KPC59B...
0001714538,The Berenstain Bears on the Moon (Bright and E...,PreSchool-Grade 2 A delightful tale told in rh...,http://ecx.images-amazon.com/images/I/A1zj84uC...


In [29]:
amazon_access_file_path = DUMP_FOLDER + "amazon_access"
book_only_candidates = fill_in_with_details(book_desc_titles,amazon_access_file_path,get_candidate,DUMP_FOLDER)

Retrieving from : ../../Project-Data/dump/book_only_candidates_with_details
It took 00:00:00.042 to get the data.


We will therefore look at the author in the API of amazon. We will match books depending on if the authors are identical. Because we do not want to crawl the API too many times we will do it for all the items that are the dictionnary ```get_candidate``` (either as key or in the value list).

In [30]:
book_only_candidates.head()

,asin,title,description,imUrl,authors,isbn,eisbn,brand,edition,publisher
2,000100039X,The Prophet,"In a distant, timeless place, a mysterious pro...",http://ecx.images-amazon.com/images/I/81ZKLPiv...,[Jane Goodall],0002113570,None,Brand: New York Dell Publishing Co Inc 1971,1st Edition,HarperCollins Distribution Services
5,0001473123,The Book of Revelation,"American Baptist pastor, Bible teacher, and wr...",http://ecx.images-amazon.com/images/I/317G8R2T...,[Sidney Sheldon],0006175015,None,Brand: HarperCollins,New Ed,HarperCollins Publishers Ltd
6,0002113570,In the Shadow of Man,"""An instant animal classic."" --Time""Apart fr...",http://ecx.images-amazon.com/images/I/51iMi0zY...,[Derek Robinson],0006473334,None,None,New edition,HarperCollins Publishers Ltd
30,0002222590,The Golden Gate,'Startlingly good...tense...ingenious' Sunday ...,http://ecx.images-amazon.com/images/I/51RjrfZe...,[Kimberley Chambers],0007503156,None,None,None,HarperCollins Publishers Ltd
66,0006176909,House of Cards,"'This blood-and-thunder tale, lifelike and tho...",http://ecx.images-amazon.com/images/I/71IaEyKy...,[Richard Chun],0060107790,None,Brand: Harper Row,1,HarperCollins


We can see that we are already doing quite well to detect candidate similars but indeed those are not the same and we now need to go further and define a suite of tools that we can use to check between those candidates.

# TESTING

*Here I put all my work in progress that didn't show any results yet but could be useful later on.*

In [28]:
from nltk.tag import StanfordNERTagger
from nltk.tokenize import word_tokenize

st = StanfordNERTagger(DATA_FOLDER+'stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz',
DATA_FOLDER+'stanford-ner/stanford-ner.jar',encoding='utf-8')

text = """
A multi-agency manhunt is under way across several states and Mexico after
police say the former Los Angeles police officer suspected in the murders of a
college basketball coach and her fiancÃ© last weekend is following through on
his vow to kill police officers after he opened fire Wednesday night on three
police officers, killing one.
"In this case, we're his target," Sgt. Rudy Lopez from the Corona Police
Department said at a press conference.
The suspect has been identified as Christopher Jordan Dorner, 33, and he is
considered extremely dangerous and armed with multiple weapons, authorities
say. The killings appear to be retribution for his 2009 termination from the
 Los Angeles Police Department for making false statements, authorities say.
Dorner posted an online manifesto that warned, "I will bring unconventional
and asymmetrical warfare to those in LAPD uniform whether on or off duty."
"""

Rudy
Lopez
Christopher
Jordan
Dorner
Dorner


In [29]:
for sentence in sent_tokenize(text):
    chunks = ne_chunk(pos_tag(word_tokenize(sentence)))
    
tokenized_text = word_tokenize(text)
classified_text = st.tag(tokenized_text)
for word,tag in classified_text:
    if(tag=='PERSON'):
        print(word)

Rudy
Lopez
Christopher
Jordan
Dorner
Dorner


### 1.1.0 - TF-IDF

In [48]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

#nltk.download('punkt') # if necessary...

stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)


def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]


def normalize(text):
    '''remove punctuation, lowercase, stem'''
    toReturn = stem_tokens(
        nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))
    return toReturn


vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')


def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0, 1]